In [ ]:
!nvcc --version
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

wersja **GPU**

In [ ]:

%%cu 
#include<stdio.h>
#define DIM 1000
struct cuComplex{
    float r;
    float i;
    //# konstruktor struktury cuComplex
    __device__ cuComplex(float a, float b):r(a),i(b){}
    //# funkcja zwracająca kwadrat magnitude
    __device__ float magnitude2(void){ return r*r+i*i; }
    //# operator mnożenia dla cuComplex
    __device__ cuComplex operator*(const cuComplex &a){ return cuComplex(r*a.r-i*a.i,i*a.r+r*a.i); }
    //# operator dodawania dla cuComplex
    __device__ cuComplex operator+(const cuComplex &a){ return cuComplex(r+a.r,i+a.i); }
};
//# funkcja julia, obliczająca fraktal julia
__device__ int julia(int x, int y){
    const float scale=1.5;
    float jx=scale*(float)(DIM/2-x)/(DIM/2);
    float jy=scale*(float)(DIM/2-y)/(DIM/2);
    cuComplex c(-0.8,0.156);
    cuComplex a(jx,jy);
    int i=0;
    for(i=0;i<200;i++){
        a=a*a+c;
        if(a.magnitude2()>1000) return 0;
    }
    return 1;
}
//# kernel wywoływany na GPU
__global__ void kernel(unsigned char *ptr){
    int x=blockIdx.x;
    int y=blockIdx.y;
    int offset=x+y*gridDim.x;

    //# Oblicz wartość fraktalu Julia dla danego punktu na ekranie
    int juliaValue=julia(x,y);
    //# Ustaw kolor piksela na czerwony (R=255, G=0, B=0) jeśli wartość fraktalu jest różna od zera
    ptr[offset*4+0]=255*juliaValue;
    ptr[offset*4+1]=0;
    ptr[offset*4+2]=0;
    //# Ustaw wartość alpha na 255 (całkowite przezroczystość)
    ptr[offset*4+3]=255;
}
//# "dev_bitmap" jest wskaźnikiem na typ "unsigned char". Wskazuje na pamięć alokowaną na karcie graficznej (GPU)
//# "DataBlock" jest rodzajem kontenera, który przechowuje informacje o pamięci alokowanej na GPU.
struct DataBlock{
    unsigned char *dev_bitmap;
};

//# Funkcja wypisz_RGB_fraktal wypisuje wartości RGB dla każdego piksela w fraktalu
void wypisz_RGB_fraktal(unsigned char *ptr){
    //# Pętla przechodząca przez wszystkie wiersze
    for(int y=0;y<DIM;y++){
        //# Pętla przechodząca przez wszystkie kolumny
        for(int x=0;x<DIM;x++){
            //# Obliczenie offsetu wskazującego na dane piksela
            int offset=x+y*DIM;
            //# Wypisanie wartości RGB dla danego piksela
            printf("\n%d, %d, %d, %d", ptr[offset*4+0], ptr[offset*4+1], ptr[offset*4+2], ptr[offset*4+3]);
        }
    }
}

//# Funkcja save_to_file zapisuje dane fraktala do pliku JPG
void save_to_file(unsigned char *ptr){
    //# Otwarcie pliku do zapisu
    FILE *fp=fopen("Fraktal_GPU.jpg","w");
    //# Zapisanie nagłówka do pliku
    fprintf(fp,"P3\n%d %d\n255\n", DIM, DIM);
    //# Pętla przechodząca przez wszystkie wiersze
     for(int y=0;y<DIM;y++){
        //# Pętla przechodząca przez wszystkie kolumny
        for(int x=0;x<DIM;x++){
            //# Obliczenie offsetu wskazującego na dane piksela
            int offset=x+y*DIM;
            //# Zapisanie wartości RGB dla danego piksela do pliku
            fprintf(fp,"\n%d %d %d", ptr[offset*4+0], ptr[offset*4+1], ptr[offset*4+2]);
        }
}
//# Zakończenie zapisu i zamknięcie pliku
fclose(fp);
}
int main(void){
    //# Pomiar czasu wykonania programu
    clock_t start, end;
    double execution_time1;
    //# Otwieranie pliku tekstowego do zapisywania czasu wykonania programu
    FILE *fp = fopen("execution_time1.txt", "w");
    for (int i = 0; i < 10; i++) {
        //# Pomiar czasu początkowego
        start = clock();
        //# Tworzenie struktury DataBlock i alokacja pamięci dla bitmapy
        DataBlock data;
        unsigned char *bitmap = (unsigned char*)malloc(DIM*DIM*4*sizeof(unsigned char));
        int image_size = DIM*DIM*4;
        //# Alokacja pamięci na GPU
        unsigned char *dev_bitmap;
        cudaMalloc((void**)&dev_bitmap, image_size);
        //# Przypisanie wskaźnika na pamięć GPU do struktury DataBlock
        data.dev_bitmap=dev_bitmap;
        //# Wywołanie jądra na GPU z wielkością siatki określoną jako DIM x DIM
        dim3 grid(DIM,DIM);
        kernel<<<grid,1>>>(dev_bitmap);
        //# Kopiowanie bitmapy z pamięci GPU do pamięci hosta
        cudaMemcpy(bitmap,dev_bitmap,image_size,cudaMemcpyDeviceToHost);
        //# Zwolnienie pamięci na GPU
        cudaFree(dev_bitmap);
        //# Zapisanie bitmapy do pliku
        save_to_file (bitmap);
        //# Pomiar czasu końcowego
        end = clock();
        //# Obliczenie czasu wykonania programu i zapisanie go do pliku
        execution_time1 = ((double)(end - start))/CLOCKS_PER_SEC;
        fprintf(fp, "%f\n", execution_time1);
        }
        //# Zamknięcie pliku tekstowego
        fclose(fp);
        return 0;
}

wersja **CPU**

In [ ]:
%%cu 
#include<stdio.h>
#define DIM 1000

struct cuComplex{
    float r;
    float i;
    //# Konstruktor inicjuje obie wartości za pomocą argumentów 'a' i 'b'
    cuComplex(float a, float b):r(a),i(b){}
    //# Funkcja 'magnitude2' oblicza i zwraca kwadrat magnitudy liczby zespolonej
    float magnitude2(void){return r*r+i*i;}
    //# Funkcje operatorowe 'operator*' i 'operator+' pozwalają na wykonywanie działań mnożenia i dodawania na liczbach zespolonych.
    cuComplex operator*(const cuComplex &a){ return cuComplex(r*a.r-i*a.i,i*a.r+r*a.i); }
    cuComplex operator+(const cuComplex &a){ return cuComplex(r+a.r,i+a.i); }
};
int julia(int x, int y){
    //# scale jest stałą, która określa skalę generowanego obrazu
    const float scale=1.5;
    //# Zmienne jx i jy są współrzędnymi punktu na płaszczyźnie zespolonej
    //# obliczane przez podzielenie różnicy pomiędzy połową wymiaru (DIM) i aktualną współrzędną (x lub y) przez połowę wymiaru (DIM/2)
    float jx=scale*(float)(DIM/2-x)/(DIM/2);
    float jy=scale*(float)(DIM/2-y)/(DIM/2);
    //# Zmienna c jest składnikiem zespolonym, który określa parametr dla funkcji julia
    cuComplex c(-0.8,0.156);
    //# Zmienna a jest składnikiem zespolonym, który określa punkt na płaszczyźnie zespolonej, dla którego liczymy wartość funkcji julia
    cuComplex a(jx,jy);
    int i=0;
    //# Jeśli wartość magnitud2 (moduł kwadratu) dla a jest większa niż 1000, funkcja zwraca 0, co oznacza, że punkt nie należy do zbioru Julia
    //# W przeciwnym razie funkcja zwraca 1, co oznacza, że punkt należy do zbioru Julia
    for(i=0;i<200;i++){
        a=a*a+c;
        if(a.magnitude2()>1000) return 0;
    }
    return 1;
}
void kernel(unsigned char *ptr){
    //# pętla po wszystkich pikselach obrazu
    for(int y=0;y<DIM;y++){
        for(int x=0;x<DIM;x++){
            //# obliczenie pozycji w tablicy `ptr` dla aktualnego piksela
            int offset=x+y*DIM;
            //# wywołanie funkcji julia() dla aktualnego piksela
            int juliaValue=julia(x,y);
            //# ustawienie koloru dla aktualnego piksela na podstawie wartości zwróconej przez funkcję julia()
            ptr[offset*4+0]=255*juliaValue;
            ptr[offset*4+1]=0;
            ptr[offset*4+2]=0;
            ptr[offset*4+3]=255;
        }
    }
}

//# "dev_bitmap" jest wskaźnikiem na typ "unsigned char". Wskazuje na pamięć alokowaną na karcie graficznej (GPU)
//# "DataBlock" jest rodzajem kontenera, który przechowuje informacje o pamięci alokowanej na GPU.
struct DataBlock{
    unsigned char *dev_bitmap;
};

//# Funkcja wypisująca wartości składowych RGB fraktala
void wypisz_RGB_fraktal(unsigned char *ptr){
    //# Dla każdej kolumny
    for(int y=0;y<DIM;y++){
        //# Dla każdego wiersza
        for(int x=0;x<DIM;x++){
            //# Obliczanie offsetu dla danego wiersza i kolumny
            int offset=x+y*DIM;
            //# Wypisywanie wartości składowych RGB
            printf("\n%d, %d, %d, %d", ptr[offset*4+0], ptr[offset*4+1], ptr[offset*4+2], ptr[offset*4+3]);
        }
    }
}

//# Funkcja zapisująca fraktal do pliku
void save_to_file(unsigned char *ptr){
    //# Otwieranie pliku do zapisu
    FILE *fp=fopen("Fraktal_CPU.jpg","w");
    //# Wpisywanie nagłówka do pliku
    fprintf(fp,"P3\n%d %d\n255\n", DIM, DIM);
    //# Dla każdej kolumny
     for(int y=0;y<DIM;y++){
        //# Dla każdego wiersza
        for(int x=0;x<DIM;x++){
            //# Obliczanie offsetu dla danego wiersza i kolumny
            int offset=x+y*DIM;
            //# Wpisywanie wartości składowych RGB do pliku
            fprintf(fp,"\n%d %d %d", ptr[offset*4+0], ptr[offset*4+1], ptr[offset*4+2]);
        }
}
fclose(fp);
}

int main(void){
    //# Pomiar czasu wykonania programu
    clock_t start, end;
    double execution_time2;
    //# Otwieranie pliku tekstowego do zapisywania czasu wykonania programu
    FILE *fp = fopen("execution_time2.txt", "w");
    //# Petla for dziala 10 razy i mierzy czas wykonywania kodu
    for (int i = 0; i < 10; i++) {
        //# start mierzenia czasu
        start = clock();
        //# Alokacja pamieci dla bitmapy
        DataBlock data;
        unsigned char *bitmap = (unsigned char*)malloc(DIM*DIM*4*sizeof(unsigned char));
        int image_size = DIM*DIM*4;
        //# Obliczenie wartosci dla fraktala za pomoca funkcji kernel()
        kernel(bitmap);
        //# Zapisanie bitmapy do pliku
        save_to_file (bitmap);
        //# koniec mierzenia czasu
        end = clock();
        //# obliczenie czasu wykonywania
        execution_time2 = ((double)(end - start))/CLOCKS_PER_SEC;
        //# zapisanie czasu w pliku
        fprintf(fp, "%f\n", execution_time2);
    }
    //# Zamykanie pliku z czasami wykonywania
    fclose(fp);
    return 0;
}

In [ ]:
!apt-get install gnuplot
!pip install matplotlib

plot dla **GPU**

In [ ]:
import matplotlib.pyplot as plt

# Read data from files
execution_time1 = []
with open("execution_time1.txt", "r") as file:
    for line in file:
        execution_time1.append(float(line.strip()))

# Create a line plot
plt.plot(execution_time1, label="Execution Time 1 DIM 1000")
plt.xlabel("Sample Number")
plt.ylabel("Execution Time (s)")
plt.legend()
plt.show()

 plot dla **CPU**

In [ ]:
import matplotlib.pyplot as plt

# Read data from files
execution_time1 = []
with open("execution_time2.txt", "r") as file:
    for line in file:
        execution_time1.append(float(line.strip()))

# Create a line plot
plt.plot(execution_time1, label="Execution Time 2 DIM 1000")
plt.xlabel("Sample Number")
plt.ylabel("Execution Time (s)")
plt.legend()
plt.show()